Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [47]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### First introduce L2 for the logistic regression model.

Build the graph:

In [5]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta * tf.nn.l2_loss(weights) 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Run it:

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.366062
Minibatch accuracy: 13.3%
Validation accuracy: 11.7%
Minibatch loss at step 500: 3.147171
Minibatch accuracy: 72.7%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.754456
Minibatch accuracy: 78.1%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.876107
Minibatch accuracy: 84.4%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.755515
Minibatch accuracy: 84.4%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 0.837598
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Minibatch loss at step 3000: 0.636931
Minibatch accuracy: 89.1%
Validation accuracy: 82.0%
Test accuracy: 89.1%


Introducing a suitable amount of regularization improves validation set and test set accuracy.
(Didn't really tune beta; merely tried out a few examples.)

#### Now let's introduce L2 to the 2-layer NN

First build the graph:

In [11]:
batch_size = 128
relu_units = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Redefine the variables reflecting NN structure.
    # First layer
    w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_units]))
    b1 = tf.Variable(tf.zeros([relu_units]))
    
    # Second layer
    w2 = tf.Variable(
    tf.truncated_normal([relu_units, num_labels]))
    b2 = tf.Variable(
    tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, w1) + b1
    relu1 = tf.nn.relu(logits1)
    logits2 = tf.matmul(relu1, w2) + b2    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + (
        beta * tf.nn.l2_loss(w1)) + (
        beta * tf.nn.l2_loss(w2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

Now run it:

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 774.281799
Minibatch accuracy: 3.9%
Validation accuracy: 22.9%
Minibatch loss at step 500: 196.180191
Minibatch accuracy: 78.1%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 114.552948
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 68.564384
Minibatch accuracy: 88.3%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.300636
Minibatch accuracy: 89.8%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.206591
Minibatch accuracy: 85.2%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 15.366088
Minibatch accuracy: 91.4%
Validation accuracy: 86.6%
Test accuracy: 93.1%


93.1%, the best results we've had so far.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [23]:
#Restrict the training data to a few batches
batch_size = 128
num_batch = 5

train_dataset_short = train_dataset[0:batch_size * num_batch + 1,]
train_labels_short = train_labels[0:batch_size * num_batch + 1,]

Now re-run the 2-layer NN, set regularization to 0, and see what happens.

In [24]:
batch_size = 128
relu_units = 1024
beta = 0

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Redefine the variables reflecting NN structure.
    # First layer
    w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_units]))
    b1 = tf.Variable(tf.zeros([relu_units]))
    
    # Second layer
    w2 = tf.Variable(
    tf.truncated_normal([relu_units, num_labels]))
    b2 = tf.Variable(
    tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, w1) + b1
    relu1 = tf.nn.relu(logits1)
    logits2 = tf.matmul(relu1, w2) + b2    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + (
        beta * tf.nn.l2_loss(w1)) + (
        beta * tf.nn.l2_loss(w2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_short.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_short[offset:(offset + batch_size), :]
        batch_labels = train_labels_short[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 355.206451
Minibatch accuracy: 9.4%
Validation accuracy: 32.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Test accuracy: 83.9%


The model quickly overfits the training set (actually 'remembers' the entire training set; shows how powerful nn can be). The model stopped improving after that and performs poorly on validation and test sets.

Now, let's try to use L2 regularization for this.

In [21]:
batch_size = 128
relu_units = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Redefine the variables reflecting NN structure.
    # First layer
    w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_units]))
    b1 = tf.Variable(tf.zeros([relu_units]))
    
    # Second layer
    w2 = tf.Variable(
    tf.truncated_normal([relu_units, num_labels]))
    b2 = tf.Variable(
    tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, w1) + b1
    relu1 = tf.nn.relu(logits1)
    logits2 = tf.matmul(relu1, w2) + b2    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + (
        beta * tf.nn.l2_loss(w1)) + (
        beta * tf.nn.l2_loss(w2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_short.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_short[offset:(offset + batch_size), :]
        batch_labels = train_labels_short[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 699.603699
Minibatch accuracy: 5.5%
Validation accuracy: 31.1%
Minibatch loss at step 500: 190.402863
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 115.473312
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 70.030281
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 42.467426
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2500: 25.755781
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 3000: 15.621137
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Test accuracy: 84.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [29]:
#Introduce dropout to the nn
batch_size = 128
relu_units = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Redefine the variables reflecting NN structure.
    # First layer
    w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_units]))
    b1 = tf.Variable(tf.zeros([relu_units]))
    
    # Second layer
    w2 = tf.Variable(
    tf.truncated_normal([relu_units, num_labels]))
    b2 = tf.Variable(
    tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, w1) + b1
    relu1 = tf.nn.dropout(tf.nn.relu(logits1), keep_prob = 0.5)
    logits2 = tf.matmul(relu1, w2) + b2    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + (
        beta * tf.nn.l2_loss(w1)) + (
        beta * tf.nn.l2_loss(w2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_short.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_short[offset:(offset + batch_size), :]
        batch_labels = train_labels_short[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 830.708923
Minibatch accuracy: 12.5%
Validation accuracy: 28.1%
Minibatch loss at step 500: 192.080704
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 116.572899
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 70.722473
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 42.902397
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 26.020548
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 3000: 15.783983
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Test accuracy: 86.1%


Improved somewhat, but not an aweful lot. Why?

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Train with the full data set and use dropout. See how good we get.

In [26]:
#Introduce dropout to the nn
batch_size = 128
relu_units = 1024
beta = 0.001

graph1 = tf.Graph()
with graph1.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Redefine the variables reflecting NN structure.
    # First layer
    w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_units]))
    b1 = tf.Variable(tf.zeros([relu_units]))
    
    # Second layer
    w2 = tf.Variable(
    tf.truncated_normal([relu_units, num_labels]))
    b2 = tf.Variable(
    tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, w1) + b1
    relu1 = tf.nn.dropout(tf.nn.relu(logits1), keep_prob = 0.5)
    logits2 = tf.matmul(relu1, w2) + b2    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + (
        beta * tf.nn.l2_loss(w1)) + (
        beta * tf.nn.l2_loss(w2))
    
    # Optimizer. Use learning rate decay.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

Train with full dataset

In [27]:
num_steps = 3001


with tf.Session(graph=graph1) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 811.678040
Minibatch accuracy: 11.7%
Validation accuracy: 23.0%
Minibatch loss at step 500: 211.032700
Minibatch accuracy: 71.1%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 125.757072
Minibatch accuracy: 70.3%
Validation accuracy: 80.6%


KeyboardInterrupt: 

91.8%: actually worse than without dropout. 
Increased training to 20k steps, 93.8% test accuracy.

Next steps, let's try:
* Use a more complex network (add one more layer)
* Use learning rate decay

Add one more layer into the nn, same activation function; 512 relu units

In [54]:
batch_size = 128
relu_units1 = 1024
relu_units2 = 512
#beta = 0

graph2 = tf.Graph()
with graph2.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Redefine the variables reflecting NN structure.
    # First layer
    w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_units1], stddev=0.01))
    b1 = tf.Variable(tf.zeros([relu_units1]))
    
    # Second layer
    w2 = tf.Variable(
    tf.truncated_normal([relu_units1, relu_units2], stddev=0.01))
    b2 = tf.Variable(
    tf.zeros([relu_units2]))
    
    # Third layer
    w3 = tf.Variable(
    tf.truncated_normal([relu_units2, num_labels], stddev=0.01))
    b3 = tf.Variable(
    tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, w1) + b1
    relu1 = tf.nn.dropout(tf.nn.relu(logits1), keep_prob = 0.5)
    logits2 = tf.matmul(relu1, w2) + b2
    relu2 = tf.nn.dropout(tf.nn.relu(logits2), keep_prob = 0.5)
    logits3 = tf.matmul(relu2, w3) + b3
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits3, tf_train_labels))
    
    # Optimizer. Use learning rate decay.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate=0.3, global_step=global_step, 
                                           decay_steps=1000, decay_rate=0.98)                                          
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2), w3) 
        + b3)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2), w3) 
        + b3)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2), w3) 
        + b3)

In [56]:
num_steps = 50001

with tf.Session(graph=graph2) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                                       feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.302039
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.394027
Minibatch accuracy: 89.1%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.568499
Minibatch accuracy: 85.2%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 0.309900
Minibatch accuracy: 90.6%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 0.313895
Minibatch accuracy: 92.2%
Validation accuracy: 87.0%
Minibatch loss at step 2500: 0.372951
Minibatch accuracy: 89.1%
Validation accuracy: 87.5%
Minibatch loss at step 3000: 0.436794
Minibatch accuracy: 86.7%
Validation accuracy: 87.6%
Minibatch loss at step 3500: 0.458944
Minibatch accuracy: 85.2%
Validation accuracy: 88.1%
Minibatch loss at step 4000: 0.388330
Minibatch accuracy: 92.2%
Validation accuracy: 88.3%
Minibatch loss at step 4500: 0.350417
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 5000: 0.423395
Minibatch accuracy: 89.8%
Validation accuracy

Loss is exploding; try smaller learning rate?
Very poor performance. Try using dropout only once? -> turns out that initializing weights with much smaller stdev worked out.

94.0%: 2 hidden layers = (1024, 256), learning rate 0.2, 10K training
94.1%: 2 hidden layers = (1024, 512), learning rate 0.2, 10K training

Next step: learning rate decay.
94.3%
95.1% !! with 20K training. Validation accurary is actually still increasing. Could get to better results if trained longer.

Next step: adjust initial weights per suggestion sqrt(2/n) -> don't seem to work?!

Training many many steps (50K): learning basically stopped after ~30K. Perhaps because the learning rate is very small by then?

## 96.8%! The best result so far
* Two hidden layers (1024, 512) with ReLu
* Dropout with 50% keep rate for both layers
* Learning rate decay
* A lot of training


